# Data preprocessing & feature engineering

**Import libraries**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from nltk.stem import PorterStemmer
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
import warnings

register_matplotlib_converters()
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 1000, 'display.max_rows', 1000, 'display.width', None)

## 1. Data collection

In [2]:
# Data Collection
health = pd.read_csv("../input/dataset/Health.csv")
holidays = pd.read_csv("../input/dataset/Holidays.csv")
data = pd.read_csv("../input/dataset/SelectedData.2.1.csv")
products = pd.read_csv("../input/dataset/SelectedProducts.2.1.csv")
samples = pd.read_csv("../input/dataset/SelectedSamples.2.1.csv")
trends = pd.read_csv("../input/dataset/Trends.csv")
weather = pd.read_csv("../input/dataset/WeatherFeatures.csv")

## 2. Data preprocessing

Merge datasets, create time series stock-keeping unit and add quantity

In [3]:
# Merge products and data to be able to extract Quantity of product groups
data = data.drop_duplicates(subset=('Pharmacy', 'Product', 'Brand', 'Strength'), keep='first')
samples_merge = pd.merge(products[['Pharmacy', 'Product', 'Brand', 'Strength']],
                         data[['Pharmacy', 'Product', 'Brand', 'Strength', 'Category', 'Description', 'Country',
                               'City']], on=['Pharmacy', 'Product', 'Brand', 'Strength'])
# Extract product group
dataframe = samples_merge[(samples_merge['Category'].str.contains('malaria', case=False)) |
                          (samples_merge['Description'].str.contains('malaria', case=False))]
# Add weather features to get time series for every stock keeping unit
dataframe = pd.merge(dataframe, weather, on=('City', 'Country'), how='left')
# Add Quantity
dataframe = pd.merge(dataframe, samples, on=(['Pharmacy', 'Product', 'Brand', 'Strength', 'Year', 'Week']), how='left')


Replace quantity of 1/2018 if the three quantities before and after are NaNs

In [4]:
# Remove quantity and frequency of time series with 2018-01 > 0 but the three quantities before and after = 0
m = (dataframe['Quantity'].notna()
     .groupby(dataframe['Pharmacy'])
     .rolling(7, center=True).sum().le(1)
     .reset_index(level='Pharmacy', drop='Pharmacy'))
dataframe['Quantity'] = dataframe['Quantity'].mask(m & dataframe['Year'].eq(2018) & dataframe['Week'].eq(1))
n = (dataframe['Frequency'].notna()
     .groupby(dataframe['Pharmacy'])
     .rolling(7, center=True).sum().le(1)
     .reset_index(level='Pharmacy', drop='Pharmacy'))
dataframe['Frequency'] = dataframe['Frequency'].mask(n & dataframe['Year'].eq(2018) & dataframe['Week'].eq(1))


Drop before first and after last occurrence for every time series

In [5]:
dataframe[['Pharmacy', 'Product', 'Brand', 'Strength']] \
    = dataframe[['Pharmacy', 'Product', 'Brand', 'Strength']].fillna('missing')  # otherwise NaNs are dropped
dataframe = dataframe.groupby(['Pharmacy', 'Product', 'Brand', 'Strength'], group_keys=False) \
    .apply(lambda x: x.loc[x['Quantity'].first_valid_index():x['Quantity'].last_valid_index()])
# replace NaN by 0 in column Quantity
dataframe['Quantity'].fillna(0, inplace=True)
dataframe['Quantity'] = dataframe['Quantity'].astype('int32')  # Quantity to integer

Data cleaning: Feature *Price* (See data-analysis.ipynb for more information)

In [6]:
# Clean prices (See data-analysis.ipynb for more information)
data = data.drop_duplicates(subset=('Pharmacy', 'Product', 'Brand', 'Strength'), keep='first')
data.loc[data['Price'] > 60000, 'Price'] = data['Cost'] * 1.2  # replace outlier with cost * 1.2
data.loc[data['Price'] == 0, 'Price'] = data['Cost'] * 1.2  # replace price 0.00 with cost * 1.2
# Add Price
dataframe = pd.merge(dataframe, data[['Pharmacy', 'Product', 'Brand', 'Strength', 'Price']],
                     on=(['Pharmacy', 'Product', 'Brand', 'Strength']), how='left')

Descriptive statistics for *Quantity*

In [7]:
dataframe['Quantity'].describe()

count    123701.000000
mean         20.039975
std         147.456589
min           0.000000
25%           0.000000
50%           0.000000
75%           3.000000
max        9012.000000
Name: Quantity, dtype: float64

Columns with missing values initially

In [8]:
dataframe.columns[dataframe.isnull().any()].tolist()

['Description',
 'City',
 'Unnamed: 0_x',
 'Year',
 'Week',
 'PrecipInt',
 'PrecipProb',
 'Humidity',
 'WindSpeed',
 'CloudCover',
 'UVIndex',
 'TempMin',
 'TempMax',
 'PrecipInt_1',
 'PrecipInt.avg_2',
 'PrecipInt.avg_5',
 'PrecipInt.avg.Lag1_5',
 'PrecipInt.5W.avg.Lag2',
 'PrecipInt.5W.avg.Lag3',
 'PrecipInt.6W.avg.Lag4',
 'PrecipInt.LookAhead_1',
 'PrecipInt.LookAhead_2',
 'PrecipProb_1',
 'PrecipProb.avg_2',
 'PrecipProb.avg_5',
 'PrecipProb.avg.Lag1_5',
 'PrecipProb.5W.avg.Lag2',
 'PrecipProb.5W.avg.Lag3',
 'PrecipProb.6W.avg.Lag4',
 'PrecipProb.LookAhead_1',
 'PrecipProb.LookAhead_2',
 'Humidity_1',
 'Humidity.avg_2',
 'Humidity.avg_5',
 'Humidity.avg.Lag1_5',
 'Humidity.avg.Lag2_5',
 'Humidity.avg.Lag3_5',
 'Humidity.avg.Lag4_6',
 'Humidity.LookAhead_1',
 'Humidity.LookAhead_2',
 'WindSpeed_1',
 'WindSpeed.avg_2',
 'WindSpeed.avg_5',
 'WindSpeed.avg.Lag1_5',
 'WindSpeed.avg.Lag2_5',
 'WindSpeed.avg.Lag3_5',
 'WindSpeed.avg.Lag4_6',
 'WindSpeed.LookAhead_1',
 'WindSpeed.LookAhead

Standardization of same Products/Brands/Strengths with different spelling

In [9]:
porterStemmer = PorterStemmer()  # function for Stemming


def stem_strings(sentence):
    tokens = sentence.split()
    stemmedTokens = [porterStemmer.stem(token) for token in tokens]
    return ' '.join(stemmedTokens)

a) Feature *Product*

In [10]:
# Product: Standardization of different spellings
uniqueProductBefore = dataframe['Product'].nunique()

dataframe['Product'] = dataframe['Product'].str.lower().str.replace("[:;.,+'-'_/%()']", " ").str.replace('\\', ' ') \
    .str.replace('[0123456789]', '').str.replace('[\s+[a-zA-Z]\s+]', '').str.replace('mg', '')
dataframe['Product'] = dataframe['Product'].apply(stem_strings)

dataframe['Product'] = dataframe['Product'].str.findall('\w{4,}').str.join(' ')  # remove words <4 characters
dataframe['Product'] = dataframe['Product'].str.replace('tablet|tab|syrup|syrp', ' ').str.replace('  ', ' ').str.strip()
dataframe['Product'].replace(to_replace=[r'[l]?a?r?te[r]?[m]?[e]?t[h]?', r'lum[aei]f[ae]n[dt]?[h]?[r]?i[mn]',
                                         r'p[io]per[a]?quin', r'sulph[au]methopyrazin[e]',
                                         r'd[iy]h[y]?droa[r]?tem[ei]sin[i]?[n]?'],
                             value=['artemether', 'lumefantrin', 'piperaquin', 'sulphamethopyrazin',
                                    'dihydroartemisinin'],
                             regex=True, inplace=True)
dataframe['Product'] = dataframe['Product'].str.replace('  ', ' ')

# print(pd.DataFrame(dataframe['Product'].unique()))
uniqueProductAfter = dataframe['Product'].nunique()
print('The number of unique products has been reduced from {0} to {1} ({2:.0f} % reduction).'
      .format(uniqueProductBefore, uniqueProductAfter,
              ((uniqueProductBefore - uniqueProductAfter) / uniqueProductBefore) * 100))

The number of unique products has been reduced from 1152 to 394 (66 % reduction).


b) Feature *Brand*

In [11]:
# Brand: Standardization of different spellings
uniqueBrandBefore = dataframe['Brand'].nunique()

dataframe['Brand'] = dataframe['Brand'].str.lower().str.replace("[:;.,+'-'_/%()']", " ").str.replace('\\', ' ') \
    .str.replace('[0123456789]', '').str.replace('mg|ml', ' ').str.replace('-', ' ').str.replace('ñ', 'n')
dataframe['Brand'] = dataframe['Brand'].str.findall('\w{4,}').str.join(' ')  # remove words shorter than 4 characters
dataframe['Brand'] = dataframe['Brand'].str.replace('tablets|tabs|syrup|syrp|susp|above', ' ') \
    .str.replace("  ", " ").str.strip()
dataframe['Brand'].replace(to_replace=[r'a?r?te[r]?[m]?[e]?[n]?t[h]?[e]?[r]?'], value=['artemeth'], regex=True,
                           inplace=True)
dataframe['Brand'].loc[dataframe['Brand'].str.contains('dabur', case=False)
                       | dataframe['Brand'].str.contains('odomos', case=False)] = 'dabur odomos'
dataframe['Brand'] = dataframe['Brand'].apply(stem_strings)
dataframe['Brand'] = dataframe['Brand'].str.replace('  ', ' ')

# print(pd.DataFrame(dataframe['Brand'].unique()))
uniqueBrandAfter = dataframe['Brand'].nunique()
print('The number of unique brands has been reduced from {0} to {1} ({2:.0f} % reduction).'
      .format(uniqueBrandBefore, uniqueBrandAfter,
              ((uniqueBrandBefore - uniqueBrandAfter) / uniqueBrandBefore) * 100))

The number of unique brands has been reduced from 1319 to 458 (65 % reduction).


c) Feature *Strength*

In [12]:
# Strength: Standardization of different spellings
uniqueStrengthBefore = dataframe['Strength'].nunique()

dataframe['Strength'] = dataframe['Strength'].str.lower().str.replace("[:.;,+'-'_/%()']", " ").str.replace('\\', ' ') \
    .str.replace('[\s+[a-zA-Z]\s+]', '')
dataframe['Strength'].replace(
    to_replace=[r'[0][ ][m]', r'[1][ ][m]', r'[1][ ][l]', r'[2][ ][m]', r'[5][ ][m]', r'[3][ ][s]',
                r'[6][ ][s]', r'[9][ ][s]', r'[12][ ][s]', r'[18][ ][s]', r'[24][ ][s]'],
    value=['0m', '1m', '1l', '2m', '5m', '3s', '6s', '9s', '12s', '18s', '24s'], regex=True, inplace=True)
dataframe['Strength'] = dataframe['Strength'].str.replace('\w{7,}', '').str.replace('litr', 'l') \
    .str.replace('mg|ml|gm|g', '')
dataframe['Strength'] = dataframe['Strength'].str.replace \
    ('al|ds|artemeth|lumefantrin|usp|bp|per|ph|int|osat|vi|tab|tablet|yellow|x|xx|xxx|actm|i|let',
     '').str.strip()
dataframe['Strength'] = dataframe['Strength'].apply(stem_strings)
dataframe['Strength'] = dataframe['Strength'].str.replace('  ', ' ')

# print(pd.DataFrame(dataframe['Strength'].unique()))
uniqueStrengthAfter = dataframe['Strength'].nunique()
print('The number of unique strengths has been reduced from {0} to {1} ({2:.0f} % reduction).'
      .format(uniqueStrengthBefore, uniqueStrengthAfter,
              ((uniqueStrengthBefore - uniqueStrengthAfter) / uniqueStrengthBefore) * 100))

The number of unique strengths has been reduced from 405 to 148 (63 % reduction).


Group time series that have been standardized

In [13]:
print('Length before grouping standardized time series:', len(dataframe))
dataframe = dataframe.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price', 'Country', 'City', 'Year', 'Week'])[
    ['Quantity', 'Frequency']].sum().reset_index()
# Drop few sales that are not assigned to product, brand and strength
dataframe = dataframe.drop(dataframe.index[(dataframe['Product'] == '') &
                                           (dataframe['Brand'] == '') &
                                           (dataframe['Strength'] == '')])

Length before grouping standardized time series: 123701


## 3. Feature engineering

3.1 Dataset-related features

a) Medical sales in this pharmacy in previous week

In [14]:
Q_ind = dataframe.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price', 'Year', 'Week'])[
    'Quantity'].sum().reset_index()

Q_ind['Quantity_individual_1'] = Q_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])['Quantity'].shift(1)
Q_ind['Quantity_individual_1'] = Q_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Quantity_individual_1'].apply(lambda x: x.fillna(x[:11].mean()))  # replace NaN with mean of following 10 weeks
Q_ind['Quantity_individual_1'] = Q_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Quantity_individual_1'].apply(lambda x: x.fillna(0))  # replace remaining NaNs with 0

b) Mean sales of this drug in this pharmacy in previous 4 weeks

In [15]:
Q_ind['Quantity_individual.avg_4'] = Q_ind['Quantity'].rolling(4).mean().shift(1)
Q_ind['Quantity_individual.avg_4'] = Q_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Quantity_individual.avg_4'].apply(lambda x: x.fillna(x[:10].mean()))  # replace NaN with mean of following 5 weeks
Q_ind['Quantity_individual.avg_4'] = Q_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Quantity_individual.avg_4'].apply(lambda x: x.fillna(0))  # replace NaN with 0

c) Mean sales of this drug in this pharmacy in previous 2 weeks

In [16]:
Q_ind['Quantity_individual.avg_2'] = Q_ind['Quantity'].rolling(2).mean().shift(1)
Q_ind['Quantity_individual.avg_2'] = Q_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Quantity_individual.avg_2'].apply(lambda x: x.fillna(x[:7].mean()))  # replace NaN with mean of following 5 weeks
Q_ind['Quantity_individual.avg_2'] = Q_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Quantity_individual.avg_2'].apply(lambda x: x.fillna(0))  # replace NaN with 0
Q_ind = Q_ind.drop(['Quantity'], axis=1)
dataframe = pd.merge(dataframe, Q_ind, on=('Pharmacy', 'Product', 'Brand', 'Strength', 'Price', 'Year', 'Week'), how='left')

d) Total medicals sales for respective product group in previous week

In [17]:
Q_total = dataframe.groupby(['Year', 'Week'])['Quantity'].sum().reset_index()
Q_total['Quantity_total_1'] = Q_total['Quantity'].shift(1)
Q_total = Q_total.drop(['Quantity'], axis=1)
Q_total = Q_total.fillna(Q_total[:11].mean())  # replace NaN with mean of the following 10 weeks
dataframe = pd.merge(dataframe, Q_total, on=('Year', 'Week'), how='left')

e) Total medical sales for respective product group in this pharmacy in previous week

In [18]:
Q_pharma = dataframe.groupby(['Pharmacy', 'Year', 'Week'])['Quantity'].sum().reset_index()
Q_pharma['Quantity_pharmacy_1'] = Q_pharma.groupby('Pharmacy')['Quantity'].shift(1)
Q_pharma['Quantity_pharmacy_1'] = Q_pharma.groupby(['Pharmacy'])['Quantity_pharmacy_1'] \
    .apply(lambda x: x.fillna(x[:11].mean()))  # replace NaN with mean of 10 following weeks
Q_pharma['Quantity_pharmacy_1'] = Q_pharma.groupby(['Pharmacy'])['Quantity'] \
    .apply(lambda x: x.fillna(x.mean()))  # replace remaining Nan with overall mean (Pharmacies with less than 11 weeks)
Q_pharma = Q_pharma.drop(['Quantity'], axis=1)
dataframe = pd.merge(dataframe, Q_pharma, on=('Pharmacy', 'Year', 'Week'), how='left')

f) Number of pharmacies with medical sales in previous week

In [19]:
Pharmacy_1 = samples.groupby(['Year', 'Week'])['Pharmacy'].nunique().reset_index()
Pharmacy_1['Pharmacy_1'] = Pharmacy_1['Pharmacy'].shift(1)
Pharmacy_1['Pharmacy_1'] = Pharmacy_1['Pharmacy_1'].fillna(method='bfill')  # fill Year 2017 Week 1 with value of Week 2
Pharmacy_1 = Pharmacy_1.drop(['Pharmacy'], axis=1)
dataframe = pd.merge(dataframe, Pharmacy_1, on=('Year', 'Week'), how='left')

g) Frequency of this medical sold in this pharmacy in previous week

In [20]:
F_ind = dataframe.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price', 'Year', 'Week'])['Frequency'].sum().reset_index()
F_ind['Frequency_individual_1'] = F_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])['Frequency'].shift(1)
F_ind['Frequency_individual_1'] = F_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Frequency_individual_1'].apply(lambda x: x.fillna(x[:11].mean()))  # replace NaN with mean of following 10 weeks
F_ind['Frequency_individual_1'] = F_ind.groupby(['Pharmacy', 'Product', 'Brand', 'Strength', 'Price'])[
    'Frequency_individual_1'].apply(lambda x: x.fillna(0))  # replace remaining NaNs with 0
F_ind = F_ind.drop(['Frequency'], axis=1)
dataframe = pd.merge(dataframe, F_ind, on=('Pharmacy', 'Product', 'Brand', 'Strength', 'Price', 'Year', 'Week'), how='left')

Rearrange dataframe

In [21]:
dataframe = dataframe[['Pharmacy', 'Product', 'Brand', 'Strength', 'Country', 'City', 'Year', 'Week', 'Quantity',
                       'Frequency', 'Quantity_individual_1', 'Quantity_individual.avg_4', 'Quantity_individual.avg_2',
                       'Quantity_total_1', 'Quantity_pharmacy_1', 'Pharmacy_1', 'Frequency_individual_1', 'Price']]

3.2 External features

a) Weather features

In [22]:
print('Minimum value for weather features:')
print(weather.loc[:, 'PrecipInt':'TempMax'].min())  # Negative values
print(weather.loc[:, 'PrecipInt_1':'PrecipInt.LookAhead_2'].min())  # Negative values
print(weather.loc[:, 'PrecipProb_1':'PrecipProb.LookAhead_2'].min())  # Negative value

weather.loc[:, 'PrecipInt':'PrecipProb'] = weather.loc[:, 'PrecipInt':'PrecipProb'].apply(
    lambda x: np.where(x < 0, 0, x))
weather.loc[:, 'PrecipInt_1':'PrecipInt.LookAhead_2'] = weather.loc[:, 'PrecipInt_1':'PrecipInt.LookAhead_2'].apply(
    lambda x: np.where(x < 0, 0, x))
weather.loc[:, 'PrecipProb_1':'PrecipProb.LookAhead_2'] = weather.loc[:, 'PrecipProb_1':'PrecipProb.LookAhead_2'].apply(
    lambda x: np.where(x < 0, 0, x))

print('\nMinimum value for weather features after cleaning:')
print(weather.loc[:, 'PrecipInt':'TempMax'].min())  # No negative values
print(weather.loc[:, 'PrecipInt_1':'PrecipInt.LookAhead_2'].min())  # No negative values
print(weather.loc[:, 'PrecipProb_1':'PrecipProb.LookAhead_2'].min())  # No negative values

dataframe = pd.merge(dataframe, weather, on=('City', 'Country', 'Year', 'Week'), how='left')

Minimum value for weather features:
PrecipInt    -3.457720e-03
PrecipProb   -1.690000e-16
Humidity      3.357143e-01
WindSpeed     1.621429e+00
CloudCover    1.660000e-06
UVIndex       0.000000e+00
TempMin       9.961429e+00
TempMax       2.043857e+01
dtype: float64
PrecipInt_1             -0.003458
PrecipInt.avg_2         -0.001729
PrecipInt.avg_5         -0.000692
PrecipInt.avg.Lag1_5    -0.000692
PrecipInt.5W.avg.Lag2   -0.000692
PrecipInt.5W.avg.Lag3   -0.000692
PrecipInt.6W.avg.Lag4   -0.000576
PrecipInt.LookAhead_1   -0.003458
PrecipInt.LookAhead_2   -0.001729
dtype: float64
PrecipProb_1             -1.690000e-16
PrecipProb.avg_2         -1.560000e-16
PrecipProb.avg_5         -1.160000e-16
PrecipProb.avg.Lag1_5    -1.160000e-16
PrecipProb.5W.avg.Lag2   -1.160000e-16
PrecipProb.5W.avg.Lag3   -1.160000e-16
PrecipProb.6W.avg.Lag4   -1.030000e-16
PrecipProb.LookAhead_1   -1.690000e-16
PrecipProb.LookAhead_2   -1.560000e-16
dtype: float64

Minimum value for weather features after clea

b) Trends features

In [23]:
# Trends features
dataframe = pd.merge(dataframe, trends, on=(['Country', 'Year', 'Week']), how='left')

c) Health features

In [24]:
# Health features
dataframe = pd.merge(dataframe, health, on=(['Country', 'Year', 'Week']), how='left')

In [25]:
# Drop year 2019 week 25 (no weather features available)
dataframe = dataframe.drop(dataframe.index[dataframe['Year'] == 2019] & dataframe.index[dataframe['Week'] == 25])
# Drop Pharmacy 309 (Yangon, Nigeria) & Pharmacy 188 (Tanta, Nigeria) (no weather features available)
dataframe = dataframe.drop(dataframe.index[dataframe['Pharmacy'] == 'Pharmacy 309'])
dataframe = dataframe.drop(dataframe.index[dataframe['Pharmacy'] == 'Pharmacy 188'])
# Drop Pharmacy 183 (Vihiga, Nigeria) (no sales and thus NaN values in multiple columns)
dataframe = dataframe.drop(dataframe.index[dataframe['Pharmacy'] == 'Pharmacy 183'])
# Drop index from merged dataframes
dataframe = dataframe.drop(['Unnamed: 0'], axis=1)
# For each time series, replace NaN in the last row of Weather LookAhead
dataframe.loc[:, 'PrecipInt':'TempMax.LookAhead_2'] = dataframe.fillna(method='ffill')

In [26]:
print('Length after grouping standardized time series:', len(dataframe))

Length after grouping standardized time series: 99807


d) Feature *Date*

In [27]:
# Feature Date
dataframe[['Year', 'Week']] = dataframe[['Year', 'Week']].astype('int32')  # change Year and Week to integer
dates = dataframe.Year * 100 + dataframe.Week
dataframe['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')

e) Feature *Holiday*

In [28]:
# Feature Holiday
holidays = holidays.drop(['HolidayType'], axis=1)
holidays['Date'] = pd.to_datetime(holidays['Date'])  # add a column for week number
holidays['Week'] = holidays['Date'].dt.strftime('%U').astype(int)
dates = holidays.Year * 100 + holidays.Week
holidays['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')  # change the format to datetime
holidays = holidays.drop_duplicates(subset=['Country', 'Date'], keep='first')  # in case of two holidays in a week
holidays = holidays.drop(['Unnamed: 0', 'Year', 'Week'], axis=1)
dataframe = pd.merge(dataframe, holidays, on=(['Country', 'Date']), how='left')
# if Holiday = NaN: change to 0, else change to 1
dataframe['Holiday'] = dataframe['Holiday'].where(dataframe['Holiday'].isnull(), 1).fillna(0).astype(int)
dataframe = dataframe.drop(['Date'], axis=1)

Columns with missing values

In [29]:
dataframe.columns[dataframe.isnull().any()].tolist()

[]

3.3 Polynomial features

In [30]:
poly = PolynomialFeatures(3)

poly1 = pd.DataFrame(poly.fit_transform(dataframe.loc[:, 'PrecipInt':'TempMax']),
                     columns=poly.get_feature_names(dataframe.loc[:, 'PrecipInt':'TempMax'].columns))

poly2 = pd.DataFrame(poly.fit_transform(dataframe.loc[:, 'Disease':'Influenza']),
                     columns=poly.get_feature_names(dataframe.loc[:, 'Disease':'Influenza'].columns))

poly3 = pd.DataFrame(poly.fit_transform(dataframe.loc[:, 'Quantity_individual_1':'Frequency_individual_1']),
                     columns=poly.get_feature_names(
                         dataframe.loc[:, 'Quantity_individual_1':'Frequency_individual_1'].columns))

dataframe = pd.concat([dataframe, poly1.loc[:, 'PrecipInt^2':'TempMax^3']], ignore_index=False, axis=1, sort=False)
dataframe = pd.concat([dataframe, poly2.loc[:, 'Disease^2':'Influenza^3']], ignore_index=False, axis=1, sort=False)
dataframe = pd.concat([dataframe, poly3.loc[:, 'Quantity_individual_1^2':'Frequency_individual_1^3']],
                      ignore_index=False, axis=1, sort=False)

3.4 Feature scaling (MinMaxScaler)

In [31]:
scaler = MinMaxScaler()
dataframe.loc[:, 'Quantity_individual_1':'Frequency_individual_1^3'] = \
    scaler.fit_transform(dataframe.loc[:, 'Quantity_individual_1':'Frequency_individual_1^3'])

3.5 One-hot encoding (*Pharmacy*, *Product*, *Brand*, *Strength*)

In [32]:
# One hot encoding for pharmacy, product, brand, strength
OneHotEncoding = pd.get_dummies(dataframe[['Pharmacy', 'Product', 'Brand', 'Strength']])
dataframe = pd.concat([dataframe, OneHotEncoding], ignore_index=False, axis=1, sort=False)

In [33]:
# Feature Date
dataframe[['Year', 'Week']] = dataframe[['Year', 'Week']].astype('int32')  # change Year and Week to integer
dates = dataframe.Year * 100 + dataframe.Week
dataframe['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')

Split the dataframe into lumpy and constant time series

In [34]:
# Split the dataframe into dataframeLumpy and dataframeConstant
a = dataframe.groupby(['Pharmacy', 'Product', 'Brand', 'Strength'])['Quantity'].describe()['50%'].reset_index()
dataframe = pd.merge(dataframe, a, on=['Pharmacy', 'Product', 'Brand', 'Strength'])
dataframe['50%'].values[dataframe['50%'].values > 0] = 1

dataframeLumpy = dataframe[dataframe['50%'] == 0]
dataframeLumpy = dataframeLumpy.reindex()
dataframeLumpy.set_index(['Date'], inplace=True)  # set index to Date

dataframeConstant = dataframe[(dataframe['50%'] == 1)]
dataframeConstant = dataframeConstant.reindex()
dataframeConstant.set_index(['Date'], inplace=True)  # set index to Date

dataframe = dataframe.reindex()
dataframe.set_index(['Date'], inplace=True)  # set index to Date

# 4. Result

In [35]:
# Size dataframe
print('Size dataframe total: {}'.format(len(dataframe)))
print('Size dataframe lumpy: {0} ({1} % of the total data)'.format(
    len(dataframeLumpy), round(len(dataframeLumpy) / len(dataframe) * 100)))
print('Size dataframe constant: {0} ({1} % of the total data)'.format(
    len(dataframeConstant), round(len(dataframeConstant) / len(dataframe) * 100)))

Size dataframe total: 99807
Size dataframe lumpy: 63423 (64 % of the total data)
Size dataframe constant: 36384 (36 % of the total data)


In [36]:
# Number of features
print('Dataset-related features:')
print('Quantity/Frequency/Pharmacy/Price:', len(dataframe.filter(regex='Quantity_|Frequency_|Pharmacy_1|Price') .columns))
print('Pharmacy (one-hot encoding):', len(dataframe.filter(regex='Pharmacy_Pharmacy').columns))
print('Product (one-hot encoding):', len(dataframe.filter(regex='Product_').columns))
print('Brand (one-hot encoding):', len(dataframe.filter(regex='Brand_').columns))
print('Strength (one-hot encoding):', len(dataframe.filter(regex='Strength_').columns))
print('\nExternal features:')
print('Weather:', len(dataframe.filter(regex='Precip|Humidity|WindSpeed|CloudCover|UVIndex|Temp').columns))
print('Trends:', len(dataframe.filter(regex='Disease|Pain|Malaria|Influenza').columns)-2)  # remove MalariaCases and MalariaDeaths from this group
print('Health/Holiday:', len(dataframe.filter(regex='MalariaCases|MalariaDeaths|AirPollution|Holiday').columns))
print('\nTotal:', len(dataframe.loc[:, 'Quantity_individual_1':'Strength_s'].columns))

Dataset-related features:
Quantity/Frequency/Pharmacy/Price: 120
Pharmacy (one-hot encoding): 231
Product (one-hot encoding): 279
Brand (one-hot encoding): 352
Strength (one-hot encoding): 144

External features:
Weather: 236
Trends: 70
Health/Holiday: 4

Total: 1436


Create CSV file

In [37]:
dataframe.to_csv('Malaria.csv')
dataframeConstant.to_csv('MalariaConstant.csv')
dataframeLumpy.to_csv('MalariaLumpy.csv')